In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def preprocess_data(X, Y):
    X = K.applications.densenet.preprocess_input(X)
    Y = K.utils.to_categorical(Y)
    return X, Y

In [ ]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
initializer = K.initializers.he_normal()
input_tensor = K.Input(shape=(32, 32, 3))

resized_images = K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(input_tensor)
model = K.applications.DenseNet201(include_top=False,
                                   weights='imagenet',
                                   input_tensor=resized_images,
                                   input_shape=(224, 224, 3),
                                   pooling='max',
                                   classes=1000)

74850304/74836368 [==============================] - 0s 0us/step


In [ ]:
for layer in model.layers:
    layer.trainable = False
output = model.layers[-1].output
flatten = K.layers.Flatten(name='feats')
output = flatten(output)
model = K.models.Model(inputs=input_tensor, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda_1 (Lambda)              (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['lambda_1[0][0]']               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                             

In [ ]:
y_test.shape

(10000, 10)

In [ ]:
preds=model.predict(x_train)
preds.shape

(50000, 1920)

In [ ]:
preds_test=model.predict(x_test)
preds_test.shape

(10000, 1920)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=42, max_iter=1000, algorithm='elkan', tol=0.000001).fit(preds_test)

preds=kmeans.predict(preds_test) # use features extracted from denseNet
preds.shape

(10000,)

In [ ]:
y1=np.argmax(y_test,axis=1)
y1.shape

(10000,)

In [ ]:
# assign predicted clusters to actual classes to get the accuracy
from collections import Counter

correct=0
mapping={}

for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices for a particular class
  p=preds[inds] # predictions made for that class
  y2=y1[inds]
  counts=dict(Counter(p))

  print("y2: ", y2[:2])
  print("counts: ", counts)
  print("----------------")

y2:  [0 0]
counts:  {4: 691, 7: 55, 6: 180, 0: 5, 1: 34, 2: 19, 8: 14, 9: 2}
----------------
y2:  [1 1]
counts:  {7: 974, 6: 8, 2: 12, 1: 1, 5: 1, 8: 2, 4: 1, 0: 1}
----------------
y2:  [2 2]
counts:  {3: 12, 2: 72, 0: 545, 1: 117, 9: 41, 8: 146, 4: 49, 5: 7, 6: 6, 7: 5}
----------------
y2:  [3 3]
counts:  {2: 790, 1: 40, 9: 72, 8: 33, 0: 10, 5: 18, 3: 13, 7: 12, 6: 8, 4: 4}
----------------
y2:  [4 4]
counts:  {1: 288, 2: 49, 3: 396, 8: 187, 6: 8, 9: 46, 0: 20, 7: 5, 4: 1}
----------------
y2:  [5 5]
counts:  {2: 398, 5: 454, 8: 88, 3: 29, 9: 17, 1: 7, 6: 2, 0: 4, 7: 1}
----------------
y2:  [6 6]
counts:  {9: 633, 1: 271, 2: 54, 4: 11, 0: 11, 3: 12, 6: 2, 8: 5, 5: 1}
----------------
y2:  [7 7]
counts:  {8: 462, 3: 406, 2: 84, 7: 11, 5: 8, 0: 4, 6: 11, 1: 9, 9: 3, 4: 2}
----------------
y2:  [8 8]
counts:  {6: 916, 9: 3, 4: 33, 8: 2, 1: 2, 2: 16, 7: 27, 0: 1}
----------------
y2:  [9 9]
counts:  {7: 953, 4: 6, 6: 29, 1: 6, 2: 4, 8: 2}
----------------


In [ ]:
#dense net
mapping={
    0:4,
    1:7,
    2:0,
    3:2,
    4:3,
    5:5,
    6:9,
    7:8,
    8:6,
    9:7
}
correct=0
for val in set(y1):
  inds=[i for i in range(len(y1)) if y1[i]==val] # all indices
  p=preds[inds]
  y2=y1[inds]
  counts=dict(Counter(p))
  correct+=counts[mapping[y2[0]]]
print("accuracy: ", correct*100/len(y1))

accuracy:  68.14
